In [1]:
import seabiscuit as sb
import pandas as pd
import threading
# import botometer
import requests
from sys import argv
import math
import queue
from sys import exit
import os.path
from datetime import datetime, timedelta

In [3]:
# %run /Users/thomaslederer/Documents/graphika_scripts/analysis_tools/get_nodes_data.ipynb


#This function takes a set of users (input as a file or taken from the graphika api)
#and splits it into 3 even batches to be separately processed in our three botometer apps.
def split_into_batches(set_of_users):
    print("...Splitting map users into botometer batches")
    map_users = {l.lower().strip() for l in set_of_users}
    q = queue.Queue()
    for u in map_users:
        q.put(u)
    print("...Queue filled. it's size is %d" %q.qsize())
    batches = [[] for i in range (3)] #batches
    rate_limit = int(math.ceil(float(len(map_users))/float(3)))
    for i in range(len(batches)):
        relevant_batch = batches[i]
        local_items_added = 0
        while local_items_added < rate_limit and not q.empty():
            #print("%d items added at index %d" %(local_items_added,i)) #DEBUG
            if len(relevant_batch) < rate_limit and not q.empty():
                relevant_batch.append(q.get())
                local_items_added+=1 
    
    print("...Out of loop. There are %d batches. " %len(batches))
    print("...Rate limit computed to be %d" %rate_limit)
    for i in range(len(batches)):
        print("...Batch index %d has %d elements." %(i,len(batches[i])))

    return batches

In [4]:
def call_botometer_pro(mk,app_auth_dict,number,batch_of_accounts):
    print("...Calling botometer pro for app number %d"   %number)
    
    botometer_api_url = 'https://botometer-pro.p.mashape.com'  #Pro-API Endpoint
    bom = botometer.Botometer(botometer_api_url=botometer_api_url,
                              wait_on_ratelimit=True,
                              mashape_key=mk,
                              **app_auth_dict)    
    
    #the strucure of the output here is a tuple of two objects, the format is
        #(handle, Json_botometer_object_response)
    i=1
    with open("botometer_%d_output.txt" %number, "w") as out:
        out.write("handle,english,universal\n")
        for handle in bom.check_accounts_in(batch_of_accounts):
            try:
                english_score=str(handle[1]['scores']['english'])
                universal_score=str(handle[1]['scores']['universal'])
                out.write (str(handle[0].strip())+","+english_score+","+universal_score+"\n")
            except KeyError as e:
                out.write(str(handle[0].strip())+",Error,Error\n")
                #print(str(i)+" accounts proccessed out of "+str(account_number)+" accounts")
                continue    
            
            i+=1
            if i%5==0:
                percentage = float(i)/float(len(batch_of_accounts))*100
                print("%d/%d accounts (%.1f%%) checked for app %d" %(i,len(batch_of_accounts),percentage,number))
            
    print("...App number %d finished processing." %number)
    
#     return results

#This fn is the main one in the program.
#It calls botometer_pro with a specified app.

In [18]:
def run_botometer():
    map_id = input('>> Enter map ID on which to run Botometer: ')
    users = pd.DataFrame(sb.Nodes(map_id)).username.values.tolist()
    results = []

    #MAIN BLOCK
    #Pro-botometer-1
    mk1 ="ST36JAjUHymshP2BwWI8DgY9Xytap1JxAGHjsnDkk6pCzipwN7"
    app_1_auth = {'consumer_key': "7dXEVaABsXMrdMWtZbrSEeL0f",'consumer_secret': "8Pc0Kb5KT4vgF1qvaZLyxFkhFNeIwDIs4QGNcC6l15PBJTwysp"}

    #Pro-botometer-2
    mk2 ="Q3kpumqc4Imsh92Lz11AkYMdJRjwp1IDmkqjsngoBhfYysMbsS"
    app_2_auth = {'consumer_key': "JDmjWf6jzGITDVshrGxwYvq9L",'consumer_secret': "XO6L4okxPxP3LWdW8GKOqSOTT8ObxkejEn5EYfFKc4RSbHPl5P"}

    #Pro-botometer-Rodrigo
    mk3 = "zfMsCJ51vmmshYkVYTeKVXxfwnubp1iS5EzjsnzKToBguaVONX"
    app_3_auth= {'consumer_key': 'LIJDUKRS8WY0b4xE4ZezybBqJ','consumer_secret': 'LUHH5Qz84C5q7hgLXzBvDyTFi6BpA1JqB9IA6OG2LqFztfM4H9'}

    botometer_batches = split_into_batches(users) #split all our users into 3 batches - to be processed in 3 separate apps
    print (botometer_batches)
    print("[*] Botometer batches: %d [*]" %len(botometer_batches))
    
    
    thread1 = threading.Thread(target=call_botometer_pro,args=[mk1,app_1_auth,1,botometer_batches[0]])
    thread1.start()
    
    
    
    thread2 = threading.Thread(target=call_botometer_pro,args=[mk2,app_2_auth,2,botometer_batches[1]])
    thread2.start()
    
    
    thread3 = threading.Thread(target=call_botometer_pro,args=[mk3,app_3_auth,3,botometer_batches[2]])
    thread3.start()
    
#     return pd.DataFrame(thread1 + thread2 + thread3)